In [4]:
import requests
from pymongo import MongoClient
import pandas as pd
import random

# Create user

In [5]:
personajes = ['Barack Obama', 'Voldemort', 'Chewbacca', 'Angela Merkel','Margherita Hack','Valentino Rossi', 
              'must4in3', 'Aretha Franklin', 'Marge Simpson', 'Darth Vader', 'Yoda', 'R2-D2', 'John Belushi',
              'Maccio Capatonda', ' Elon Musk', 'Che Guevara', 'Gollum', 'Samwise Gamgee', 'Jon Snow','Arya Stark',
              'Heidi', 'Hermione', 'Pikachu', 'Topolino', 'Berlusconi', 'Greta Thunberg','Hulk', 'Dilan Dog', 
              'Stephen Hopkins','Oprah Winfre', 'Frida Kahlo','Tyrion Lannister', 'Jennifer Aniston']

for personage in personajes:
    res = requests.get(f'http://localhost:3000/user/create/{personage}')

In [6]:
print(res)

<Response [200]>


In [7]:
res.json()

{'data': {'_id': {'$oid': '5f0cdde9cf27f22204086ae2'},
  'username': 'Jennifer Aniston'},
 'status': 'ok'}

In [8]:
# check users in Mongo
dbName = "api_chat_sentiment_analysis"
mongodbURL = f"mongodb://localhost/{dbName}"
client = MongoClient(mongodbURL, connectTimeoutMS=2000, serverSelectionTimeoutMS=2000)
db = client.get_database()

In [9]:
cur = db.user.find({"username":{"$ne": None}})

In [10]:
df = pd.DataFrame(cur)
df.head()

,_id,username
0,5f0cdde8cf27f22204086ac2,Barack Obama
1,5f0cdde8cf27f22204086ac3,Voldemort
2,5f0cdde8cf27f22204086ac4,Chewbacca
3,5f0cdde8cf27f22204086ac5,Angela Merkel
4,5f0cdde8cf27f22204086ac6,Margherita Hack


# Create Chat

In [11]:
df["_id"]= df["_id"].astype(str) 

In [12]:
five_chats = ['How I met your mother', 'Dumbledore’s Army', 
              'Tesla SpaceX','Italian restaurants in Madrid','datamad0620']
chats_room = dict()
for i,chat in enumerate(five_chats):   
    chats_room[f'{chat}']= {'user_ids': [random.choice([row for row in df['_id']]) for _ in range(5)]}

In [13]:
for room,user_ids in chats_room.items():
    res = requests.get(f'http://localhost:3000/chat/create/{room}?user_id={user_ids.values()}')

In [14]:
print(res)

<Response [200]>


In [15]:
res.json()

{'chat_users': 4,
 'create chat status': 'Ok',
 'create users status': 'Some user_id did not exist in the database, or you tried insert the same id many times',
 'data': {'_id': {'$oid': '5f0cddebcf27f22204086ae7'},
  'chat_name': 'datamad0620',
  'users_ids': ['5f0cdde8cf27f22204086acb',
   '5f0cdde8cf27f22204086ad4',
   '5f0cdde8cf27f22204086ad2',
   '5f0cdde8cf27f22204086ac7']}}

In [16]:
cur = db.chatItem.find({"chat_name":{"$ne": None}})

In [17]:
df_chat = pd.DataFrame(cur)
df_chat.head()

,_id,chat_name,users_ids
0,5f0cddeacf27f22204086ae3,How I met your mother,"[5f0cdde8cf27f22204086ad6, 5f0cdde8cf27f222040..."
1,5f0cddebcf27f22204086ae4,Dumbledore’s Army,"[5f0cdde9cf27f22204086ae1, 5f0cdde8cf27f222040..."
2,5f0cddebcf27f22204086ae5,Tesla SpaceX,"[5f0cdde9cf27f22204086ae2, 5f0cdde8cf27f222040..."
3,5f0cddebcf27f22204086ae6,Italian restaurants in Madrid,"[5f0cdde8cf27f22204086ad8, 5f0cdde8cf27f222040..."
4,5f0cddebcf27f22204086ae7,datamad0620,"[5f0cdde8cf27f22204086acb, 5f0cdde8cf27f222040..."


# Add user in a Chat

In [38]:
df_chat["_id"]= df_chat["_id"].astype(str) 

In [39]:
user_random = random.choice([row for row in df['_id']])
conversation_random = random.choice([row for row in df_chat['_id']])

In [40]:
res = requests.get(f'http://localhost:3000/chat/{conversation_random}/adduser?user_id={user_random}')

In [41]:
print(res)

<Response [200]>


In [42]:
res.json()

{'chat_name': 'datamad0620',
 'users_ids': ['5f0cdde8cf27f22204086acb',
  '5f0cdde8cf27f22204086ad4',
  '5f0cdde8cf27f22204086ad2',
  '5f0cdde8cf27f22204086ac7',
  '5f0cdde8cf27f22204086adc']}

# Add message in chat

kaggle dataset quote
https://www.kaggle.com/coolcoder22/quotes-dataset

In [43]:
quote = pd.read_csv('inputs/quote.csv', encoding = 'latin-1')

In [44]:
quote

,Auther,quote
0,A. A. Milne,"If you live to be a hundred, I want to live to..."
1,A. A. Milne,Promise me you'll always remember: You're brav...
2,A. A. Milne,"Did you ever stop to think, and forget to star..."
3,A. A. Milne,Organizing is what you do before you do someth...
4,A. A. Milne,"Weeds are flowers too, once you get to know them."
...,...,...
36160,Zsa Zsa Gabor,"I know nothing about sex, because I was always..."
36161,Zsa Zsa Gabor,Conrad Hilton was very generous to me in the d...
36162,Zsa Zsa Gabor,The women's movement hasn't changed my sex lif...
36163,Zsa Zsa Gabor,I always liked parties. You meet people; you c...


In [142]:
conversations = []
for _ in range(100):
    conversations.append({random.choice([row for row in df['_id']]): random.choice([row for row in quote['quote']])})

In [143]:
conversation_random = random.choice([row for row in df_chat['_id']])

In [144]:
for dic in conversations:
    for key,values in dic.items():
        res = requests.get(f'http://localhost:3000/chat/{conversation_random}/addmessage?user_id={key}&text={values}')

In [145]:
print(res)

<Response [500]>


In [125]:
res.json()

{'message': "Error. user_id doesn't exist in this chat", 'status': 'api-error'}

# Get messages list from a chat

In [146]:
conversation_random = random.choice([row for row in df_chat['_id']])

In [147]:
res = requests.get(f'http://localhost:3000/chat/{conversation_random}/list')

In [148]:
print(res)

<Response [200]>


In [149]:
res.json()

{'chat_name': 'Italian restaurants in Madrid',
 'conversation_id': {'$oid': '5f0cddebcf27f22204086ae6'},
 'messages': [{'message': 'The key to success is to focus our conscious mind on things we desire not things we fear.',
   'user_id': '5f0cdde8cf27f22204086acf',
   'username': 'Maccio Capatonda'},
  {'message': 'It is better to have loved and lost than never to have lost at all.',
   'user_id': '5f0cdde8cf27f22204086ac2',
   'username': 'Barack Obama'},
  {'message': 'Either systems are in balance or they are falling apart. If people are acting in what appears to be a twisted way, I want to know the reason for that.',
   'user_id': '5f0cdde8cf27f22204086ad8',
   'username': 'Pikachu'},
  {'message': "Acting with Denzel is like playing tennis with someone that's better than you. You either play better tennis or get blown off the court.",
   'user_id': '5f0cdde8cf27f22204086ac6',
   'username': 'Margherita Hack'},
  {'message': 'I would never say never to returning to comedy.',
   'us

# Get sentiment analysis

NLP & Text Sentiment Analysis:
​
- [https://www.nltk.org/]
- [https://towardsdatascience.com/basic-binary-sentiment-analysis-using-nltk-c94ba17ae386]
- [https://www.digitalocean.com/community/tutorials/how-to-perform-sentiment-analysis-in-python-3-using-the-natural-language-toolkit-nltk]

In [150]:
conversation_random = random.choice([row for row in df_chat['_id']])

In [151]:
res = requests.get(f'http://localhost:3000/chat/{conversation_random}/sentiment')

In [152]:
res.json()

[{'sentence': 'The key to success is to focus our conscious mind on things we desire not things we fear.',
  'sentiment': {'neg': 0.0, 'neu': 0.624, 'pos': 0.376, 'compound': 0.8413}},
 {'sentence': 'It is better to have loved and lost than never to have lost at all.',
  'sentiment': {'neg': 0.205, 'neu': 0.491, 'pos': 0.304, 'compound': 0.4939}},
 {'sentence': 'Either systems are in balance or they are falling apart.',
  'sentiment': {'neg': 0.151, 'neu': 0.849, 'pos': 0.0, 'compound': -0.1531}},
 {'sentence': 'If people are acting in what appears to be a twisted way, I want to know the reason for that.',
  'sentiment': {'neg': 0.0, 'neu': 0.929, 'pos': 0.071, 'compound': 0.0772}},
 {'sentence': "Acting with Denzel is like playing tennis with someone that's better than you.",
  'sentiment': {'neg': 0.0, 'neu': 0.581, 'pos': 0.419, 'compound': 0.7351}},
 {'sentence': 'You either play better tennis or get blown off the court.',
  'sentiment': {'neg': 0.0, 'neu': 0.629, 'pos': 0.371, 'co

# Get reccomender analysis

In [153]:
user_random = random.choice([row for row in df['_id']])

In [154]:
res = requests.get(f'http://localhost:3000/user/{user_random}/recommend')

In [155]:
print(res)

<Response [200]>


In [156]:
res.json()

{'Me': {'Margherita Hack': 1.0},
 'Recommended users': {'Berlusconi': 0.74224128002284,
  'Che Guevara': 0.7510979742805717,
  'Jennifer Aniston': 0.7434144169622005,
  'John Belushi': 0.72603642310951,
  'Valentino Rossi': 0.7582168118772317}}